# MLP Classification with CR Dataset
<hr>

We will build a text classification model using Multi Layer Perceptron on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from tensorflow.keras.preprocessing.text import Tokenizer
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')


Bad key text.latex.unicode in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
h

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


,sentence,label,split
0,"simplistic , silly and tedious .",0,train
1,"it 's so laddish and juvenile , only teenage b...",0,train
2,exploitative and largely devoid of the depth o...,0,train
3,garbus discards the potential for pathological...,0,train
4,a visually flashy but narratively opaque and e...,0,train
...,...,...,...
10657,both exuberantly romantic and serenely melanch...,1,train
10658,mazel tov to a film about a family 's joyous l...,1,train
10659,standing in the shadows of motown is the best ...,1,train
10660,it 's nice to see piscopo again after all thes...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5331,5331
1,5331,5331


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing: Word2Vec Static
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

## Load Pre-trained Word Embedding: Word2Vec

__1. Load `Word2Vec` Pre-trained Word Embedding__

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

In [8]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

In [10]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [11]:
oov_tok = '<OOV>'

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


## Define `clean_doc` function
__2. Define a function to clean the document called __`clean_doc()`____

In [12]:
def clean_doc(sentences, word_index):
    clean_sentences = []
    for sentence in sentences:
        sentence = sentence.lower().split()
        clean_word = []
        for word in sentence:
            if word in word_index:
                clean_word.append(word)
        clean_sentence = ' '.join(clean_word)
        clean_sentences.append(clean_sentence)
    return clean_sentences

In [13]:
clean_sentences = clean_doc(sentences, word_index)
clean_sentences[0:3]

['simplistic silly and tedious',
 "it 's so laddish and juvenile only teenage boys could possibly find it funny",
 'exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable']

## Define `sentence_to_avg` function
__3. Define a `sentence_to_avg` function__

We will use this function to calculate the mean of word embedding representation.

In [14]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = (sentence.lower()).split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(word2vec.word_vec('i').shape)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    count = 0
    for w in words:
        if w in word_to_vec_map:
            total += word_to_vec_map.word_vec(w)
            count += 1
            
    if count!= 0:
        avg = total/count
    else:
        avg
    return avg

In [15]:
i = word2vec.word_vec('i')[0]
print(word2vec.word_vec('i')[0])
j = word2vec.word_vec('am')[0]
print(word2vec.word_vec('am')[0])
k = word2vec.word_vec('handsome')[0]
print(word2vec.word_vec('handsome')[0])
mean = (i+j+k)/3
print('the mean of word embedding is: ', mean)

-0.22558594
-0.16699219
0.11376953
the mean of word embedding is:  -0.09293619791666667


In [16]:
# Example of the functions used in a sentence
mysentence = 'I am handsome'
sentence_to_avg(mysentence, word2vec)

array([-0.0929362 ,  0.03125   , -0.03914388,  0.09879557,  0.07088598,
        0.03092448, -0.00651042, -0.04437256,  0.08068848,  0.07242838,
        0.00160726, -0.10530599, -0.07389323, -0.08854166,  0.00565592,
        0.15136719, -0.0460612 ,  0.19482422,  0.1101888 ,  0.05924479,
       -0.18457031,  0.00716146,  0.16153972,  0.02437337, -0.01578776,
        0.06119792, -0.25048828,  0.02799479,  0.0853475 , -0.14029948,
        0.13688152, -0.01350911, -0.05493164, -0.01090495,  0.03352864,
        0.09635416, -0.04239909,  0.00777181, -0.1438802 ,  0.06510416,
        0.14560954, -0.11295573,  0.25520834,  0.08833822,  0.14339192,
        0.037028  , -0.02832031, -0.00139872,  0.00309245, -0.17871094,
        0.06852213,  0.07910156,  0.09513346,  0.11425781, -0.00488281,
        0.11051432, -0.01139323, -0.08479818, -0.09277344, -0.03263346,
       -0.00374349,  0.07977295, -0.26416016, -0.05135091,  0.06111654,
       -0.06933594, -0.06486002,  0.18766277, -0.04826609,  0.03

## Encode Sentence into Word2Vec Representation

In [17]:
def encoded_sentences(sentences):

    encoded_sentences = []

    for sentence in sentences:

        encoded_sentence = sentence_to_avg(sentence, word2vec)
        encoded_sentences.append(encoded_sentence)

    encoded_sentences = np.array(encoded_sentences)
    return encoded_sentences

In [18]:
embedded_sentences = encoded_sentences(clean_sentences)
print(embedded_sentences.shape)
embedded_sentences

(10662, 300)


array([[ 0.22493489, -0.03442383, -0.05810547, ..., -0.02913412,
        -0.1258138 ,  0.20052083],
       [ 0.02938334,  0.03513813,  0.02398427, ..., -0.06489563,
         0.06098684,  0.04986   ],
       [ 0.02773868,  0.08043715,  0.01731962, ..., -0.06427361,
         0.0566128 , -0.00628303],
       ...,
       [ 0.04141097,  0.06443648,  0.04822679, ..., -0.02214883,
         0.04628129, -0.03073085],
       [ 0.03426514,  0.04058971, -0.00080566, ..., -0.01333466,
         0.07069702, -0.06862793],
       [ 0.04723011, -0.04642001,  0.03718844, ...,  0.01513117,
         0.04043302,  0.05668502]])

# MLP Model: 1

## Model Definition

In [19]:
def define_model(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=50, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [20]:
model_0 = define_model(300)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                15050     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 15,101
Trainable params: 15,101
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [21]:
'''
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True
'''
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [22]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Training 1: 
Epoch 1/40
300/300 [==============================] - 2s 7ms/step - loss: 0.5913 - accuracy: 0.6941 - val_loss: 0.5200 - val_accuracy: 0.7507
Epoch 2/40
300/300 [==============================] - 1s 4ms/step - loss: 0.5009 - accuracy: 0.7612 - val_loss: 0.4832 - val_accuracy: 0.7648
Epoch 3/40
300/300 [==============================] - 1s 5ms/step - loss: 0.4791 - accuracy: 0.7704 - val_loss: 0.4781 - val_accuracy: 0.7629
Epoch 4/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4726 - accuracy: 0.7701 - val_loss: 0.4685 - val_accuracy: 0.7694
Epoch 5/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4615 - accuracy: 0.7793 - val_loss: 0.4856 - val_accuracy: 0.7591
Epoch 6/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4582 - accuracy: 0.7823 - val_loss: 0.4639 - val_accuracy: 0.7694
Epoch 7/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4531 - accuracy: 0.7777 - val_loss: 0.4613 - val_accuracy: 

300/300 [==============================] - 2s 7ms/step - loss: 0.4312 - accuracy: 0.7962 - val_loss: 0.4713 - val_accuracy: 0.7777
Epoch 15/40
300/300 [==============================] - 2s 5ms/step - loss: 0.4265 - accuracy: 0.7968 - val_loss: 0.4663 - val_accuracy: 0.7814
Epoch 00015: early stopping
Test Accuracy: 78.42401266098022

Training 4: 
Epoch 1/40
300/300 [==============================] - 2s 6ms/step - loss: 0.5953 - accuracy: 0.6984 - val_loss: 0.5236 - val_accuracy: 0.7486
Epoch 2/40
300/300 [==============================] - 1s 4ms/step - loss: 0.5045 - accuracy: 0.7509 - val_loss: 0.5026 - val_accuracy: 0.7589
Epoch 3/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4818 - accuracy: 0.7634 - val_loss: 0.4831 - val_accuracy: 0.7570
Epoch 4/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4716 - accuracy: 0.7690 - val_loss: 0.4799 - val_accuracy: 0.7608
Epoch 5/40
300/300 [==============================] - 1s 4ms/step - loss: 0.4630 - 

300/300 [==============================] - 1s 2ms/step - loss: 0.4552 - accuracy: 0.7824 - val_loss: 0.4743 - val_accuracy: 0.7692
Epoch 7/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4501 - accuracy: 0.7801 - val_loss: 0.4704 - val_accuracy: 0.7711
Epoch 8/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4474 - accuracy: 0.7843 - val_loss: 0.4722 - val_accuracy: 0.7711
Epoch 9/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4425 - accuracy: 0.7886 - val_loss: 0.4801 - val_accuracy: 0.7674
Epoch 10/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4401 - accuracy: 0.7889 - val_loss: 0.4714 - val_accuracy: 0.7702
Epoch 11/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4402 - accuracy: 0.7930 - val_loss: 0.4786 - val_accuracy: 0.7664
Epoch 12/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4342 - accuracy: 0.7928 - val_loss: 0.4825 - val_accuracy: 0.7598
Epoch 13/40
30

300/300 [==============================] - 1s 2ms/step - loss: 0.4365 - accuracy: 0.7883 - val_loss: 0.4651 - val_accuracy: 0.7795
Epoch 14/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4303 - accuracy: 0.7938 - val_loss: 0.4652 - val_accuracy: 0.7786
Epoch 15/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4296 - accuracy: 0.7921 - val_loss: 0.4665 - val_accuracy: 0.7711
Epoch 16/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4281 - accuracy: 0.7904 - val_loss: 0.4743 - val_accuracy: 0.7730
Epoch 17/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4258 - accuracy: 0.7952 - val_loss: 0.4646 - val_accuracy: 0.7758
Epoch 18/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4223 - accuracy: 0.7971 - val_loss: 0.4661 - val_accuracy: 0.7871
Epoch 19/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4237 - accuracy: 0.8009 - val_loss: 0.4658 - val_accuracy: 0.7786
Epoch 20/40

Test Accuracy: 78.23639512062073

Training 10: 
Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.5923 - accuracy: 0.6881 - val_loss: 0.5348 - val_accuracy: 0.7223
Epoch 2/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4982 - accuracy: 0.7586 - val_loss: 0.5116 - val_accuracy: 0.7373
Epoch 3/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4790 - accuracy: 0.7698 - val_loss: 0.5010 - val_accuracy: 0.7420
Epoch 4/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4681 - accuracy: 0.7779 - val_loss: 0.4936 - val_accuracy: 0.7514
Epoch 5/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4613 - accuracy: 0.7789 - val_loss: 0.4907 - val_accuracy: 0.7552
Epoch 6/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4551 - accuracy: 0.7789 - val_loss: 0.4887 - val_accuracy: 0.7589
Epoch 7/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4501 - accuracy: 0.7839 

## Summary

In [23]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,77.881914,78.81912,78.424013,77.485931,79.080677,77.298313,78.799248,78.705442,78.236395,76.360226,78.109128


In [24]:
report = record
report = report.to_excel('Emb_MLP_MR.xlsx', sheet_name='model_1')

<hr>

# MLP Model: 2

## Model Definition

In [25]:
def define_model_2(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [26]:
model_1 = define_model_2(300)
model_1.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 101       
Total params: 30,201
Trainable params: 30,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [27]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [28]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Obtain the word to index
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_2(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.5790 - accuracy: 0.7084 - val_loss: 0.5096 - val_accuracy: 0.7488
Epoch 2/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4885 - accuracy: 0.7676 - val_loss: 0.4828 - val_accuracy: 0.7601
Epoch 3/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4726 - accuracy: 0.7720 - val_loss: 0.4756 - val_accuracy: 0.7704
Epoch 4/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4614 - accuracy: 0.7800 - val_loss: 0.4731 - val_accuracy: 0.7751
Epoch 5/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4552 - accuracy: 0.7806 - val_loss: 0.4709 - val_accuracy: 0.7732
Epoch 6/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4500 - accuracy: 0.7855 - val_loss: 0.4677 - val_accuracy: 0.7741
Epoch 7/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4445 - accuracy: 0.7882 - val_loss: 0.4684 - val_accuracy: 0.7732
Epoch 

300/300 [==============================] - 1s 2ms/step - loss: 0.4346 - accuracy: 0.7896 - val_loss: 0.4515 - val_accuracy: 0.7730
Epoch 11/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4315 - accuracy: 0.7955 - val_loss: 0.4539 - val_accuracy: 0.7692
Epoch 12/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4275 - accuracy: 0.7942 - val_loss: 0.4539 - val_accuracy: 0.7664
Epoch 13/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4269 - accuracy: 0.7949 - val_loss: 0.4546 - val_accuracy: 0.7636
Epoch 14/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4220 - accuracy: 0.8019 - val_loss: 0.4585 - val_accuracy: 0.7598
Epoch 15/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4206 - accuracy: 0.8010 - val_loss: 0.4483 - val_accuracy: 0.7795
Epoch 16/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4164 - accuracy: 0.8047 - val_loss: 0.4536 - val_accuracy: 0.7692
Epoch 17/40

300/300 [==============================] - 1s 2ms/step - loss: 0.3925 - accuracy: 0.8149 - val_loss: 0.4886 - val_accuracy: 0.7805
Epoch 24/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3893 - accuracy: 0.8158 - val_loss: 0.4852 - val_accuracy: 0.7795
Epoch 25/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3843 - accuracy: 0.8215 - val_loss: 0.4871 - val_accuracy: 0.7777
Epoch 26/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3830 - accuracy: 0.8203 - val_loss: 0.4891 - val_accuracy: 0.7711
Epoch 27/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3792 - accuracy: 0.8234 - val_loss: 0.4890 - val_accuracy: 0.7758
Epoch 28/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3747 - accuracy: 0.8245 - val_loss: 0.4917 - val_accuracy: 0.7824
Epoch 29/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3737 - accuracy: 0.8261 - val_loss: 0.4904 - val_accuracy: 0.7758
Epoch 30/40

300/300 [==============================] - 1s 2ms/step - loss: 0.3873 - accuracy: 0.8234 - val_loss: 0.4606 - val_accuracy: 0.7767
Epoch 28/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3810 - accuracy: 0.8251 - val_loss: 0.4656 - val_accuracy: 0.7814
Epoch 29/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3809 - accuracy: 0.8262 - val_loss: 0.4642 - val_accuracy: 0.7777
Epoch 30/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3742 - accuracy: 0.8278 - val_loss: 0.4637 - val_accuracy: 0.7739
Epoch 31/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3715 - accuracy: 0.8334 - val_loss: 0.4675 - val_accuracy: 0.7814
Epoch 32/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3703 - accuracy: 0.8348 - val_loss: 0.4649 - val_accuracy: 0.7795
Epoch 00032: early stopping
Test Accuracy: 78.79924774169922

Training 8: 
Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.573

Epoch 4/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4583 - accuracy: 0.7815 - val_loss: 0.4803 - val_accuracy: 0.7580
Epoch 5/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4546 - accuracy: 0.7836 - val_loss: 0.4756 - val_accuracy: 0.7617
Epoch 6/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4479 - accuracy: 0.7838 - val_loss: 0.4716 - val_accuracy: 0.7627
Epoch 7/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4435 - accuracy: 0.7870 - val_loss: 0.4713 - val_accuracy: 0.7636
Epoch 8/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4357 - accuracy: 0.7912 - val_loss: 0.4731 - val_accuracy: 0.7617
Epoch 9/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4354 - accuracy: 0.7910 - val_loss: 0.4738 - val_accuracy: 0.7608
Epoch 10/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4358 - accuracy: 0.7892 - val_loss: 0.4753 - val_accuracy: 0.7570
Epoch

## Summary

In [29]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,77.975631,77.22587,77.954972,77.673548,78.236395,79.080677,78.799248,78.236395,77.673548,77.110696,77.996698


In [30]:
report = record2
report = report.to_excel('Emb_MLP_MR_2.xlsx', sheet_name='model_2')

<hr>

# MLP Model: 3

## Model Definition

In [31]:
def define_model_3(input_length=300):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense( units=100, activation='relu', input_shape=(input_length,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=50, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense( units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [32]:
model_2 = define_model_3(300)
model_2.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 51        
Total params: 35,201
Trainable params: 35,201
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [33]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=1, 
                                             mode='auto', restore_best_weights=True)

In [34]:
# Parameter Initialization
oov_tok = "<UNK>"
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

sentences, labels = list(corpus.sentence), list(corpus.label)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the data into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)
    
    # Define the word_index
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)
    word_index = tokenizer.word_index
    
    # Clean the sentences
    Xtrain = clean_doc(train_x, word_index)
    Xtest = clean_doc(test_x, word_index)

    # Encode the sentences into word embedding average representation
    Xtrain = encoded_sentences(Xtrain)
    Xtest = encoded_sentences(Xtest)
    
    # Define the input shape
    model = define_model_3(Xtrain.shape[1])

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))
    print()

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 


C:\Users\Diardano Raihan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.5882 - accuracy: 0.6751 - val_loss: 0.4625 - val_accuracy: 0.7798
Epoch 2/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4958 - accuracy: 0.7585 - val_loss: 0.4491 - val_accuracy: 0.7882
Epoch 3/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4811 - accuracy: 0.7677 - val_loss: 0.4403 - val_accuracy: 0.7919
Epoch 4/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4686 - accuracy: 0.7731 - val_loss: 0.4447 - val_accuracy: 0.7873
Epoch 5/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4632 - accuracy: 0.7762 - val_loss: 0.4431 - val_accuracy: 0.7891
Epoch 6/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4544 - accuracy: 0.7807 - val_loss: 0.4419 - val_accuracy: 0.7891
Epoch 7/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4485 - accuracy: 0.7831 - val_loss: 0.4392 - val_accuracy: 0.7938
Epoch 

300/300 [==============================] - 1s 2ms/step - loss: 0.4275 - accuracy: 0.7965 - val_loss: 0.4422 - val_accuracy: 0.7861
Epoch 12/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4221 - accuracy: 0.8017 - val_loss: 0.4415 - val_accuracy: 0.7814
Epoch 13/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4121 - accuracy: 0.8097 - val_loss: 0.4571 - val_accuracy: 0.7889
Epoch 14/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4107 - accuracy: 0.8119 - val_loss: 0.4578 - val_accuracy: 0.7777
Epoch 15/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4003 - accuracy: 0.8185 - val_loss: 0.4497 - val_accuracy: 0.7758
Epoch 16/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3987 - accuracy: 0.8176 - val_loss: 0.4489 - val_accuracy: 0.7899
Epoch 17/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3886 - accuracy: 0.8209 - val_loss: 0.4560 - val_accuracy: 0.7749
Epoch 18/40

300/300 [==============================] - 1s 3ms/step - loss: 0.3539 - accuracy: 0.8389 - val_loss: 0.5310 - val_accuracy: 0.7730
Epoch 00023: early stopping
Test Accuracy: 78.42401266098022

Training 6: 
Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.5861 - accuracy: 0.6788 - val_loss: 0.4874 - val_accuracy: 0.7730
Epoch 2/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4974 - accuracy: 0.7608 - val_loss: 0.4732 - val_accuracy: 0.7852
Epoch 3/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4783 - accuracy: 0.7672 - val_loss: 0.4613 - val_accuracy: 0.7767
Epoch 4/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4671 - accuracy: 0.7767 - val_loss: 0.4628 - val_accuracy: 0.7739
Epoch 5/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4565 - accuracy: 0.7804 - val_loss: 0.4635 - val_accuracy: 0.7730
Epoch 6/40
300/300 [==============================] - 1s 2ms/step - loss: 0.4522 - a

300/300 [==============================] - 1s 2ms/step - loss: 0.3477 - accuracy: 0.8455 - val_loss: 0.5336 - val_accuracy: 0.7570
Epoch 25/40
300/300 [==============================] - 1s 3ms/step - loss: 0.3401 - accuracy: 0.8451 - val_loss: 0.5429 - val_accuracy: 0.7598
Epoch 26/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3448 - accuracy: 0.8462 - val_loss: 0.5258 - val_accuracy: 0.7523
Epoch 27/40
300/300 [==============================] - 1s 2ms/step - loss: 0.3298 - accuracy: 0.8525 - val_loss: 0.5293 - val_accuracy: 0.7561
Epoch 28/40
300/300 [==============================] - 1s 3ms/step - loss: 0.3287 - accuracy: 0.8518 - val_loss: 0.5457 - val_accuracy: 0.7598
Epoch 00028: early stopping
Test Accuracy: 76.73546075820923

Training 9: 
Epoch 1/40
300/300 [==============================] - 1s 3ms/step - loss: 0.5986 - accuracy: 0.6758 - val_loss: 0.5165 - val_accuracy: 0.7458
Epoch 2/40
300/300 [==============================] - 1s 3ms/step - loss: 0.4959

## Summary

In [35]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,80.41237,77.788192,78.986865,76.547843,78.424013,78.517824,78.330207,76.735461,77.298313,80.112571,78.315366


In [36]:
report = record3
report = report.to_excel('Emb_MLP_MR_3.xlsx', sheet_name='model_3')